<a href="https://colab.research.google.com/github/DeepHMS/voila-barplot-tool/blob/main/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import io
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
import base64
from datetime import datetime

# --- Predefined CSV Data (for testing) ---
DEMO_CSV = """Category,Value
A,10
B,15
C,7
D,12
E,9"""

# --- Widgets ---
file_uploader = widgets.FileUpload(
    accept='.csv', multiple=False, description='Upload CSV'
)
load_demo_button = widgets.Button(description='Load Demo Data', button_style='info')
x_dropdown = widgets.Dropdown(description='X Column')
y_dropdown = widgets.Dropdown(description='Y Column')
color_picker = widgets.ColorPicker(value='blue', description='Bar Color')
title_text = widgets.Text(value='My Bar Plot', description='Title')
xlabel_text = widgets.Text(value='X-axis', description='X Label')
ylabel_text = widgets.Text(value='Y-axis', description='Y Label')
dpi_slider = widgets.IntSlider(value=100, min=50, max=300, step=10, description='DPI')
plot_button = widgets.Button(description='Draw Plot', button_style='success')
download_button = widgets.Button(description='Download Plot', button_style='info')
output = widgets.Output()

df = None  # Global variable to store uploaded DataFrame

# --- Functions ---
def on_file_upload(change):
    global df
    if change['new']:
        uploaded_file = next(iter(change['new'].values()))
        content = uploaded_file['content']
        df = pd.read_csv(io.BytesIO(content))
        update_dropdowns()
    else:
        with output:
            clear_output()
            print("Please upload a CSV file.")

def on_load_demo(_):
    global df
    df = pd.read_csv(io.StringIO(DEMO_CSV))
    update_dropdowns()

def update_dropdowns():
    if df is not None:
        columns = df.columns.tolist()
        x_dropdown.options = columns
        y_dropdown.options = columns
        x_dropdown.value = columns[0] if columns else None
        y_dropdown.value = columns[1] if len(columns) > 1 else None
        with output:
            clear_output()
            print("CSV loaded successfully! Select X and Y columns to draw the plot.")
    else:
        with output:
            clear_output()
            print("No data available.")

def draw_plot(_):
    if df is not None and x_dropdown.value and y_dropdown.value:
        with output:
            clear_output()
            fig, ax = plt.subplots(figsize=(8, 5))
            ax.bar(df[x_dropdown.value], df[y_dropdown.value], color=color_picker.value)
            ax.set_title(title_text.value)
            ax.set_xlabel(xlabel_text.value)
            ax.set_ylabel(ylabel_text.value)
            plt.show()
    else:
        with output:
            clear_output()
            print("Please upload a file and select columns.")

def download_plot(_):
    if df is not None and x_dropdown.value and y_dropdown.value:
        buf = io.BytesIO()
        fig, ax = plt.subplots(figsize=(8, 5))
        ax.bar(df[x_dropdown.value], df[y_dropdown.value], color=color_picker.value)
        ax.set_title(title_text.value)
        ax.set_xlabel(xlabel_text.value)
        ax.set_ylabel(ylabel_text.value)
        plt.savefig(buf, format='png', dpi=dpi_slider.value)
        buf.seek(0)

        # Create base64-encoded data URL for download
        b64 = base64.b64encode(buf.getbuffer()).decode()
        filename = f"bar_plot_{datetime.now().strftime('%Y%m%d_%H%M%S')}.png"
        with output:
            clear_output()
            display(widgets.HTML(
                f'<a href="data:image/png;base64,{b64}" download="{filename}">'
                f'Click here to download {filename}</a>'
            ))
            print("Click the link to download the plot.")
    else:
        with output:
            clear_output()
            print("Nothing to download.")

# --- Event Listeners ---
file_uploader.observe(on_file_upload, names='value')
load_demo_button.on_click(on_load_demo)
plot_button.on_click(draw_plot)
download_button.on_click(download_plot)

# --- Layout ---
ui = widgets.VBox([
    widgets.HBox([file_uploader, load_demo_button]),
    widgets.HBox([x_dropdown, y_dropdown]),
    color_picker,
    title_text, xlabel_text, ylabel_text,
    dpi_slider,
    widgets.HBox([plot_button, download_button]),
    output
])

display(ui)